# Aufgabe 6 (Scala: Mandelbrot)

_Verwenden Sie ausschließlich aus der Lehrveranstaltung bekannte Scala-Konstrukte, dabei sind `var` und seiteneffektbehaftete Methoden/Klassen/Objekte strikt untersagt._

<div class="alert alert-block alert-danger">
<b>WICHTIG: Die folgende Zelle (<tt>import $ivy...</tt>) muss unbedingt <i>vor allen anderen</i> ausgeführt werden!</b>
</div>

In [ ]:
import $ivy.`org.scala-lang.modules::scala-parallel-collections:1.0.4`
import scala.collection.parallel.CollectionConverters._
import scala.collection.parallel.immutable._

Eine Komplexe Zahl $c$ ist genau dann Element der Mandelbrot-Menge, wenn die Mandelbrot-Folge $z_{n+1} = z_n^2 + c$ für sie beschränkt (⇝ Teilaufgabe **b)**: `bounded`) bleibt $(z \in \mathbb{C}, z_0 = 0)$.

In [ ]:
case class Complex(real: Double, img: Double)
val maxIter : Int = 100
val z0 = Complex(0,0)

def abs: (Complex) => Double = /* ... // Betrag           */ {case Complex(real,img) => math.sqrt(real*real + img*img)}
def add: (Complex, Complex) => Complex = /* ... // Summe  */ {case (Complex(ra,ia), Complex(rb,ib)) => Complex(ra+rb,ia+ib)}
def square: Complex => Complex = /* ... // Quadrat        */ {case Complex(real,img) => Complex(real*real-img*img,2*real*img)}

**a)** Vervollständigen Sie die Funktion `mSeq`. Sie berechnet die (unendliche) Mandelbrot-Folge für einen gegebenen Wert von $c$.
```scala
  mSeq(Complex(0,1)) = LazyList(Complex(0, 0), Complex(0, 1),
     Complex(-1, 1), Complex(0, -1), ...) 
```

In [ ]:
def mSeq: Complex => LazyList[Complex] = c => {
   def helper: /*** IHR CODE HIER (1) ***/ = z =>
      /*** IHR CODE HIER (2) ***/
   helper( /*** IHR CODE HIER (3) ***/ )
}

In [ ]:
// Beispiel und Schnelltest:
assert(mSeq(Complex(0,1)).take(4) == LazyList(Complex(0, 0), Complex(0, 1),Complex(-1, 1), Complex(0, -1))); print("✅")

<details>
<summary><i><span style='background:#b2ebf2'>Hinweis</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

Die Funktion `mSeq` bekommt lediglich die komplexe Zahl `c` als Parameter, die in der Formel der Mandelbrot-Folge eine _Konstante_ ist. Die Mandelbrot-Folge selbst beginnt für jedes `c` hingegen stets bei $z_0$. Anschließend müssen nach und nach noch alle weiteren $z_{n+1}$ jeweils aus der vorangehenden Zahl $z_n$ und $c$ gemäß Formel berechnet werden: $z_{n+1} \leftarrow z_n^2 + c$.

Dazu bietet es sich an, eine "innere" _rekursive_ Hilfsfunktion `helper` zu definieren. Die Konstante `c` bleibt als Teil der Umgebung ("Closure" von `mSeq`) dauerhaft zur Verfügung. Daher genügt es, wenn `helper` jeweils nur die vorangehende Zahl `z` (entspricht $z_n$) vom Typ `Complex` entgegen nimmt und die `LazyList` aller folgenden Zahlen mit `z` als Kopfelement erzeugt.
- Damit ist die Signatur von `helper` (erste Zeile) einfach.
- Die Implementierung von `helper` (mittlere Zeile) besteht im rekursiven Aufbau der `LazyList`.
- Die Hilfsfunktion `helper` muss anfangs (letzte Zeile) nur noch geeignet aufgerufen werden.

</div>

<details>
<summary><i><span style='background:#c8e6c9'>Lösungsvorschlag 1: Signatur</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
def helper: Complex => LazyList[Complex] = z => // ... 
```

</div>
</details>

<details>
<summary><i><span style='background:#c8e6c9'>Lösungsvorschlag 2: Implementierung</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
z #:: helper(add(square(z),c)) 
```

</div>
</details>

<details>
<summary><i><span style='background:#c8e6c9'>Lösungsvorschlag 3: initialer Aufruf</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
helper(z0) 
```

</div>
</details>
</details>

<details>
<summary><i><span style='background:#c8e6c9'>Lösungsvorschlag</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
def mSeq: Complex => LazyList[Complex] = c => {
   def helper: Complex => LazyList[Complex] = z =>
      z #:: helper(add(square(z),c))
   helper(z0)
} 
```

</div>
</details>

**b)** Vervollständigen Sie nun die Funktion `bounded`. Sie ermittelt, ob die Mandelbrot-Folge für einen gegebenen Wert von $c$ beschränkt bleibt. Hierzu sollen nur die ersten `maxIter` Elemente der Folge betrachtet werden. Wenn der Betrag mindestens eines Elements größer als 2 ist, gilt die Folge als nicht beschränkt.
```scala
  bounded(Complex(0, 1)) = true     bounded(Complex(1, 0)) = false 
```

In [ ]:
def bounded: Complex => Boolean = c =>
   /*** IHR CODE HIER ***/

In [ ]:
// Beispiel und Schnelltest:
assert(bounded(Complex(0, 1)) == true); print("✅")
assert(bounded(Complex(1, 0)) == false); print("✅")

<details>
<summary><i><span style='background:#b2ebf2'>Hinweis</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

- Zunächst benötigen wir die Mandelbrot-Folge für den Wert `c`. Diese liefert uns die vorangehend implementierte Funktion `mSeq`.
- Von dieser Folge benötigen wir lediglich die ersten `maxIter` Elemente. Dazu können wir die Funktion `take` von `LazyList` verwenden.
- Für jedes Element $z$ dieser Teilfolge ist zu prüfen, ob $|z| > 2$ ist. Dafür bietet sich die Funktion (höherer Ordnung) `exists` von `LazyList` an.
- Die an `exists` zu übergebende (anonyme) Funktion bauen wir aus der vorgegebenen Funktion `abs`.

</div>
</details>

<details>
<summary><i><span style='background:#c8e6c9'>Lösungsvorschlag</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
def bounded: Complex => Boolean = c =>
   !mSeq(c).take(maxIter).exists(abs(_) > 2) 
```

</div>
</details>

**c)** Vervollständigen Sie die Funktion `computeSet`, die für eine Liste von Werten <ins>**parallel**</ins> bestimmt, ob sie Element der Mandelbrot-Menge sind und eine Liste der Ergebnisse zurückgibt.
```scala
  computeSet(List(Complex(0, 0), Complex(0, 1), Complex(1, 0))) =
     List(true, true, false) 
```

In [ ]:
def computeSet: List[Complex] => List[Boolean] = in =>
   /*** IHR CODE HIER ***/

In [ ]:
// Beispiel und Schnelltest:
assert(computeSet(List(Complex(0, 0), Complex(0, 1), Complex(1, 0))) == List(true, true, false)); print("✅")

<details>
<summary><i><span style='background:#b2ebf2'>Hinweis</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

Die Funktion `computeSet` bekommt die Liste `in` der zu untersuchenden komplexen Zahlen und soll für jede davon bestimmen, ob sie Element der Mandelbrot-Menge (also "beschränkt") ist. Dazu bietet sich die Funktion höherer Ordnung `map` der Scala-List zusammen mit der vorangehend implementierten Funktion `bounded` an.

Da die Berechnung **<ins>parallel</ins>** erfolgen muss, können wir z.B. mittels `.par`/`.toList` die übergebene Liste `in` zunächst "parallelisieren" (`.par`), dann `map` aufrufen und das Ergebnis anschließend wieder in eine sequentielle Liste zurückwandeln (`.toList`).

</div>
</details>

<details>
<summary><i><span style='background:#c8e6c9'>Lösungsvorschlag</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
def computeSet: List[Complex] => List[Boolean] = in =>
   in.par.map(bounded).toList 
```

</div>
</details>

**d)** Vervollständigen Sie die Funktion `mSum`, die die Summe genau der Werte aus einer Liste berechnet, die Elemente der Mandelbrot-Menge sind. Verwenden Sie hierfür `computeSet`.
```scala
  mSum(List(Complex(0, 0), Complex(0, 1), Complex(1, 0))) = Complex(0, 1) 
```

In [ ]:
def mSum: List[Complex] => Complex = in => {
   // Entferne Elemente, die nicht in MM sind:
   val mSet = in.zip( /*** IHR CODE HIER (1) ***/ ).filter( /*** IHR CODE HIER (2) ***/ ).map( /*** IHR CODE HIER (3) ***/ )
   mSet.foldLeft /*** IHR CODE HIER (4) ***/
}

In [ ]:
// Beispiel und Schnelltest:
assert(mSum(List(Complex(0, 0), Complex(0, 1), Complex(1, 0))) == Complex(0, 1)); print("✅")

<details>
<summary><i><span style='background:#b2ebf2'>Allgemeiner Hinweis zum Lösungsansatz</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

Für diese Aufgabe gibt es verschiedene Lösungsmöglichkeiten. Allerdings "zwingt" uns der vorgedruckte Code zur Verwendung der vorgegebenen Funktionen höherer Ordnung.

Die Funktion `mSum` erhält eine Liste `in` von komplexen Zahlen - z.B.: `in` = $(c_1, c_2, c_3, c_4, c_5, \ldots, c_n)$

Daraus soll sie (laut Kommentar) alle Elemente "entfernen", die **nicht** Teil der Mandelbrot-Menge sind - oder anders ausgedrückt: Sie soll die Liste `mSet` aller komplexen Zahlen aus `in` _"behalten"_, die tatsächlich Teil der Mandelbrot-Menge sind - z.B.: `mSet` = $(c_2, c_5, \ldots, c_k)$

Dazu dient die Kette aus `zip`/`filter`/`map` zusammen mit den bereits vorangehend implementierten Funktionen - z.B. `computeSet(in)`...

Anschließend kann diese Liste `mSet` mittels Faltung (`foldLeft`) zur gewünschten Summe akkumuliert werden - z.B.: `add`(`add`(`add`($\ldots$(`add`(`z0`,$c_2$),$c_5$),$\ldots$),$c_k$)

</div>
</details>

<details>
<summary><i><span style='background:#b2ebf2'>Hinweis zu (1)</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

Die Funktion `mSum` erhält eine Liste `in` von komplexen Zahlen - z.B.: `in` = $(c_1, c_2, c_3, c_4, c_5, \ldots, c_n)$

Mittels der vorangehend implementierten Funktion `computeSet(in)` können wir nun zu jeder Zahl $c_i$ bestimmen, ob sie Teil der Mandelbrot-Menge ist - z.B.:
- `computeSet(in) = (false, true, false, false, true, ..., false)`

Mit `in.zip(computeSet(in)))` erhalten wir die Liste der Tupel $(c_i,b_i)$ so, dass $b_i$ genau dann `true` ist, wenn $c_i$ Teil der Mandelbrot-Menge (also _beschränkt_) ist - z.B.:
- `((c1,false), (c2,true), (c3,false), (c4,false), (c5,true), ..., (cn,false))`

</div>
</details>

<details>
<summary><i><span style='background:#b2ebf2'>Hinweis zu (2)</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

Nach dem Aufruf von `zip` haben wir die Liste der Tupel $(c_i,b_i)$ so, dass $b_i$ genau dann `true` ist, wenn $c_i$ Teil der Mandelbrot-Menge (also _beschränkt_) ist - z.B.:
- `((c1,false), (c2,true), (c3,false), (c4,false), (c5,true), ..., (cn,false))`

Mittels `filter(_._2)` können wir aus dieser Liste alle Tupel extrahieren, bei denen das zweite Tupel-Element (`_._2`) wie gewünscht `true` ist, d.h. $c_i$ Teil der Mandelbrot-Menge (also _beschränkt_) ist - z.B.:
- `((c2,true), (c5,true), ...)`

</div>
</details>

<details>
<summary><i><span style='background:#b2ebf2'>Hinweis zu (3)</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

Nach dem Aufruf von `filter` haben wir die Liste der Tupel $(c_i,true)$ so, dass _**alle**_ $c_i$ darin Teil der Mandelbrot-Menge (also _beschränkt_) sind - z.B.:
- `((c2,true), (c5,true), ...)`

Davon benötigen wir aber nur die komplexen Zahlen $c_i$ selbst, also die jeweils _ersten_ Tupel-Elemente. Dazu dient der abschließende Aufruf von `map(_._1)`, der aus jedem $(c_i,true)$ nur das $c_i$ liefert - z.B.:
- `(c2, c5, ...)`

</div>
</details>

<details>
<summary><i><span style='background:#b2ebf2'>Hinweis zu (4)</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

Nach dem Aufruf von `map` haben wir in `mSet` die Liste _genau derjenigen_ komplexen Zahlen $c_i$ aus der Liste `in`, die Teil der Mandelbrot-Menge (also _beschränkt_) sind - z.B.:
- `(c2, c5, ...)`

Diese müssen nur noch zusammen addiert werden, wozu der `foldLeft`-Aufruf dient:
- Das "neutrale Element" der Addition ist die vorgegebene komplexe Zahl `z0`.
- Die "Operation" ist die ebenfalls vorgegebene Funktion `add`.
- Damit ergibt sich die letzte Zeile zu: `mSet.foldLeft(z0)(add)`

</div>
</details>

<details>
<summary><i><span style='background:#c8e6c9'>Lösungsvorschlag</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
def mSum: List[Complex] => Complex = in => {
   // Entferne Elemente, die nicht in MM sind:
   val mSet = in.zip(computeSet(in)).filter(_._2).map(_._1)
   mSet.foldLeft(z0)(add)
} 
```

</div>
</details>